# Usage

The following shows how you can implement this pretty easily. Installation is fairly simple through github (PyPI incoming).

In [ ]:
!pip install git+https://github.com/jhaberbe/chinese_restaurant_process

  Cloning https://github.com/jhaberbe/crp to /private/var/folders/2n/j06nrn2n7r524t776sngh0xr0000gr/T/pip-req-build-u3m05wha
  Running command git clone --filter=blob:none --quiet https://github.com/jhaberbe/crp /private/var/folders/2n/j06nrn2n7r524t776sngh0xr0000gr/T/pip-req-build-u3m05wha
  Resolved https://github.com/jhaberbe/crp to commit b35ae312829e2391ae6ecbdbf1169f2cd8858fc6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


# Using the module

Really, the front facing portion is the ChineseRestaurantProcess object. Just pull that out, load your matrix that you want to perform inference on, and then ".run".

You can use a subset of the dataset (1/50th in my experience), and still get fairly robust results!

In [ ]:
import scanpy as sc
from crp.dirichlet import ChineseRestaurantProcess

adata = sc.read_h5ad("/Users/jameshaberberger/GitHub/chinese-restaurant-process/data/adata.h5ad")
adata = adata[adata.X.sum(axis=1) > 100]

crp = ChineseRestaurantProcess(adata[::100].X.todense())
crp.run(epochs=10)

100%|██████████| 1232/1232 [00:02<00:00, 424.26it/s]


# Label Transfer

Once you've predicted the classes, just assign labels using the predict function. I've included a minimum membership argument, so that you can remove classes with low membership from inference. However, if you wanted, you could always just set it to zero, and get the ful bayesian nonparametric inference experience..

In [7]:
# Predict labels for the full dataset
# You can adjust the `min_membership` parameter to control the minimum membership threshold for assigning labels.
# 1% works well for me. 0% is true CRP, but expect small clusters, which may or may not be representative of reality.
adata.obs["labels"] = crp.predict(adata.X.todense(), min_membership=0.01)

100%|██████████| 123130/123130 [01:04<00:00, 1896.44it/s]
/var/folders/2n/j06nrn2n7r524t776sngh0xr0000gr/T/ipykernel_72946/558921049.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["labels"] = crp.predict(adata.X.todense(), min_membership=0.01)
